In [4]:
import csv
import pandas as pd
import random
import pyarrow.feather as feather
import regex as re
from cleantext import clean
import pyarrow.feather as feather
from multiprocessing import Pool
import gc
import nltk
import itertools
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import *
import time
import gc

In [5]:
#Indhenter et sample på ca. 1,45M artikler, meget tidseffektivt
def getSample(csvstring: str):
    random.seed(0)
    n = 11000000 #number of records in file (excludes header)
    s = 1900000 #desired sample size
    skip = sorted(random.sample(range(1,n+1),n-s))
    # Read the CSV file, skipping the randomly selected rows
    sampled_data = pd.read_csv(csvstring, on_bad_lines='skip', skiprows=skip, index_col=0)
    sampled_data = sampled_data.reset_index()
    return sampled_data


In [17]:
sampled_data = getSample('data/news_cleaned_2018_02_13-1.csv')


C:\Users\olink\AppData\Local\Temp\ipykernel_7520\2548950878.py:7: DtypeWarning: Columns (0,1,11,14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  sampled_data = pd.read_csv(csvstring, on_bad_lines='skip', skiprows=skip)


In [18]:

len(sampled_data)


1472768

In [8]:
sampled_data.head()

,index,Unnamed: 0,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source
0,0,11,18,blackgenocide.org,conspiracy,http://blackgenocide.org/speaking_request.html,\n\nSpeaking Engagement Request\n\n\n\nContact...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Request Speaking Engagement,NaN,NaN,[''],NaN,NaN,NaN,NaN
1,1,12,19,blackgenocide.org,conspiracy,http://blackgenocide.org/archived_articles/neg...,"""…I have set before you life and death, blessi...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,BlackGenocide.org,NaN,NaN,"['planned parenthood', 'minorities', 'black cu...",The Negro Project is a devastating eugenic sta...,NaN,NaN,NaN
2,2,17,24,awm.com,unreliable,http://awm.com/woman-waves-hand-in-front-of-li...,Most people’s pets are a member of the family ...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"Woman Waves Hand In Front Of Lizard’s Tank, Ca...",Wendy Michaels,NaN,[''],NaN,NaN,NaN,NaN
3,3,18,25,awm.com,unreliable,http://awm.com/man-goes-to-throw-away-christma...,Dogs are fantastic. They provide so much joy i...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Man Goes To Throw Away Christmas Tree. Doesn’t...,Kate Anslinger,NaN,[''],NaN,NaN,NaN,NaN
4,4,34,41,beforeitsnews.com,fake,http://beforeitsnews.com/politics/2018/01/poll...,Poll: Calls for War From Israelis and Palestin...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Poll: Calls for War From Israelis and Palestin...,NaN,NaN,[''],NaN,NaN,NaN,NaN


In [19]:
#Fjerner de artikler vi ikke skal bruge, og klassificerer alt som reliable/fake
def removeLabels(df): 
    #klassificerer alle de artikler vi vil bruge ind i reliable eller fake
    df.type = df.type.replace({'political': 'reliable', 'junksci': 'fake', 'bias' : 'fake', 'satire': 'fake', 'conspiracy': 'fake', 'rumor': 'fake', 'unreliable' : 'fake', 'clickbait': 'fake', 'hate': 'fake'})
    #fjerner artikler som har volapyk types (inklusiv 'unknown')
    df = df[(df.type == 'reliable') |(df.type == 'fake')]
    #fjerner artikler som ikke har nogen type
    df = df[df.type.notnull()]
    #fjerner artikler uden content
    df = df[df.content.notnull()]
    #fjerner duplerede artikler, ud over en enkelt
    df = df.drop_duplicates(subset = 'content', keep = 'last')
    #reset index gør, at hvis vi fjerner artikle [2], bliver artikel [3] rykket ned på index [2] osv. dernedad.
    df = df.reset_index()
    return df 

#Returnerer et dictionary, som repræsenterer den procentmæssige fordeling af typerne i et dataframe
def labelperc(df): 
    labeldict = {}
    for i in df.type: 
        if i in labeldict: 
            labeldict[i] +=1
        else: 
            labeldict[i] = 1
    for i in labeldict: 
        labeldict[i] = labeldict[i]/len(df)*100
    return labeldict
cleanSample = removeLabels(sampled_data)
print(labelperc(cleanSample)) 
print(len(cleanSample))




{'fake': 51.66993688660282, 'reliable': 48.33006311339719}
1082496


In [10]:
del sampled_data
gc.collect()
#fjerner sampled_data, som vi ikke bruger længere, fra memory

0

In [11]:
cleanSample.content[5]

'Animal rescue workers in California received an unusual call near the end of December 2017. In a dumpster, bystanders found a very weird-looking animal. And when they arrived at the scene to help it out, they found that it was a female bear cub that was utterly hairless after losing it all. Now that the cub, who has since been named Eve, is under the care of the Bear League rescue group, which is in Homewood, California, she is getting the care she needs to get her health back on track.\n\nThe rescue group said that female cub lost her hair because of a bad case of mange. And when she was taken in, she was rushed to emergency care for an entire week. And at the end of it, they saved her life. But if she hadn’t been taken to the Bear League rescue group, she might not have made it.\n\n“We’ve seen this condition before, it seems to result from the cub’s mother being killed early in the season and then the very small young one tries desperately to make it on their own but goes hungry mos

In [12]:
#Bare et check for om der er dublerede artikler. Denne funktion er ikke længere vigtig, fordi den altid returnerer [], efter et dataframe
#har været igennem removeLabels(). Ellers ville den returnere en liste af dubleret content
def checkForDuplicateArticles(df): 
    articledict = {}
    df_sample = df.sample(frac=0.1, random_state=1)
    for i in df_sample.content: 
        if str(i) in articledict: 
            articledict[str(i)] +=1
        else: 
            articledict[str(i)] = 1
    #at this point we have all different contents, and the amount of times we see them
    articledict = {key:val for key, val in articledict.items() if val >1}
    #here we delete the contents which we only see once

    contentsToRemove = list(articledict.keys())
    #get a list of the content, we will remove articles for containing
    
    
    return contentsToRemove
    
wordstoremove = checkForDuplicateArticles(cleanSample)
len(wordstoremove)




0